# Library Import

In [1]:
from pprint import pprint
from collections import deque
from math import ceil
from time import sleep, time
import pickle
import os

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns


from warnings import filterwarnings
filterwarnings('ignore')

# Data Load

In [2]:
player_stats = pd.read_csv('data/results/2020_player_stats.csv')
player_stats

,created_at,player,death_type,time_survived(s),dbnos,total_distance,headshot_kills,rank,win,tournament_id,...,defender_damage/30m,n_of_type1,n_of_type2,survival_rate,type1_num_of_attack,type2_num_of_attack,attacker_survival_rate,type1_num_of_defense,type2_num_of_defense,defender_survival_rate
0,2020-11-22T18:23:25Z,TSM_MiracU,byplayer,1448.394,4,4210.94530,1,6,False,eu-pcs3gf,...,166.934872,4.0,6.0,0.900000,2.0,0.0,1.0,2.0,6.0,0.875
1,2020-11-22T18:23:25Z,TSM_Fexx,byplayer,1426.132,0,3547.17340,1,6,False,eu-pcs3gf,...,0.000000,4.0,3.0,0.857143,3.0,2.0,1.0,1.0,1.0,0.500
2,2020-11-22T18:23:25Z,TSM_vard,byplayer,996.121,1,2912.90070,0,6,False,eu-pcs3gf,...,91.073273,2.0,1.0,0.666667,1.0,1.0,1.0,1.0,0.0,0.000
3,2020-11-22T18:23:25Z,TSM_Iroh,byplayer,1425.539,1,3343.79850,0,6,False,eu-pcs3gf,...,0.000000,1.0,4.0,0.800000,0.0,1.0,1.0,1.0,3.0,0.750
4,2020-11-22T18:23:25Z,ETG_forever,byplayer,1411.228,0,4583.47940,0,7,False,eu-pcs3gf,...,0.000000,1.0,1.0,0.500000,0.0,1.0,1.0,1.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39413,2020-05-18T07:32:55Z,HC_Free,byzone,949.159,0,920.51697,0,11,False,as-pcs1kr,...,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.000
39414,2020-05-18T07:32:55Z,1IP_KidAsh,byplayer,705.843,0,1241.75010,0,14,False,as-pcs1kr,...,128.268725,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500
39415,2020-05-18T07:32:55Z,1IP_Jaemin,byplayer,705.855,0,1609.51880,0,14,False,as-pcs1kr,...,0.000000,1.0,1.0,0.500000,0.0,1.0,1.0,1.0,0.0,0.000
39416,2020-05-18T07:32:55Z,1IP_Lildara,byplayer,695.705,0,1152.51610,0,14,False,as-pcs1kr,...,0.000000,1.0,1.0,0.500000,0.0,0.0,0.0,1.0,1.0,0.500


In [3]:
with open('./data/standard_intervals/collcection_of_intervals.pickle', 'rb') as f:
    collcection_of_intervals = pickle.load(f)

# 필요 함수

In [4]:
def compare_interval(row, col):
    
    standard = collcection_of_intervals[col] # 기준값
    measure = row[col] # 측정값
    
    for i in range(10):
        a, b = standard[i]
        if measure < a:
            return i+0.5
        if measure >=a and measure<=b:
            return i+1
    return 10
        
def to_point_10(df, col):
    df[col+'_점수'] = df.apply(lambda row: compare_interval(row, col), axis=1)
    
def compute_summation(df):
    
    # 운영
    df['투척물_점수'] = df[['습득량_점수', '소비량_점수', '소비량/습득량_점수']].mean(axis=1)
    df['거리_점수'] = df[['displacement_점수', 'total_distance_점수', 'shooting_distance_점수']].mean(axis=1)
    df['호전성_점수'] = df[['num_of_attack/30m_점수', 'attacker_num_of_attack/30m_점수', 'defender_num_of_attack_점수']].mean(axis=1)
    df['운영_점수'] = df[['투척물_점수', '거리_점수', '호전성_점수']].mean(axis=1)
    
    # 전투
    df['교전_전투력_점수'] = df[['kill(30m)_점수', 'win_rate_점수', 'damage/30m_점수', 'acc_점수']].mean(axis=1)
    df['attacker_전투력_점수'] = df[['attacker_kill(30m)_점수', 'attacker_win_rate_점수', 'attacker_damage/30m_점수', 'attacker_acc_점수']].mean(axis=1)
    df['defender_전투력_점수'] = df[['defender_kill(30m)_점수', 'defender_win_rate_점수', 'defender_damage/30m_점수', 'defender_acc_점수']].mean(axis=1)
    df['전투_점수'] = df[['교전_전투력_점수', 'attacker_전투력_점수', 'defender_전투력_점수']].mean(axis=1)
    
    # 사격
    df['사격_점수'] = df[['acc_점수', 'attacker_acc_점수', 'defender_acc_점수', 'headshot_kills(30m)_점수', '확정킬_비율_점수']].mean(axis=1)
    
    # 생존력
    df['생존력_점수'] = df[['survival_rate_점수', 'attacker_survival_rate_점수', 'defender_survival_rate_점수', 'time_survived(s)_점수']].mean(axis=1)
    
    # 최종 점수
    df['최종 점수'] = df[['운영_점수', '전투_점수', '사격_점수', '생존력_점수']].mean(axis=1)
    
    # 운영 제외
    # df['최종 점수2'] = df[['전투_점수', '사격_점수', '생존력_점수']].mean(axis=1)
    
    # 최종 순위
    df['최종 순위'] = df['최종 점수'].rank(method='min', ascending=False)

# 활용 컬럼

In [5]:
cols = ['num_of_attack/30m', 'attacker_num_of_attack/30m', 'defender_num_of_attack',   # 호전성
        'total_distance', 'displacement', 'shooting_distance',                         # 거리
        '습득량', '소비량', '소비량/습득량',                                           # 투척물
        'headshot_kills(30m)', '확정킬_비율', 'acc', 'attacker_acc', 'defender_acc',   # 사격
        'kill(30m)', 'win_rate', 'damage/30m',                                         # 교전 전투력
        'attacker_kill(30m)', 'attacker_win_rate', 'attacker_damage/30m',              # attacker 전투력             
        'defender_kill(30m)', 'defender_win_rate', 'defender_damage/30m',              # defender 전투력 
        'survival_rate', 'attacker_survival_rate', 'defender_survival_rate', 'dbnos', 'time_survived(s)' # 생존력
       ]

In [6]:
column_renames = \
{'tournament_id':'토너먼트ID' ,
 'created_at':'경기 시작 시간',
 'player':'선수ID',
 'time_survived(s)':'생존시간',
 'dbnos':'DBNOs',
 'total_distance':'총 이동거리',
 'headshot_kills':'헤드샷 킬수',
 'rank':'경기 순위',
 'num_of_attack/30m':'사격(횟수/30분)',
 'defender_num_of_attack':'Defender 교전 공격(횟수)',
 'attacker_num_of_attack/30m':'선공(횟수/30분)',
 'displacement':'교전 이동거리',
 'shooting_distance':'사격거리',
 '습득량':'투척물 습득량',
 '소비량':'투척물 소비량',
 '소비량/습득량':'투척물 소비율',
 'headshot_kills(30m)':'헤드샷(킬수/30분)',
 '확정킬_비율':'확정킬 비율',
 'acc':'명중률',
 'attacker_acc':'Attacker 명중률',
 'defender_acc':'Defender 명중률',
 'kill(30m)':'킬(수/30분)',
 'win_rate':'승률',
 'damage/30m':'데미지(피해량/30분)',
 'attacker_kill(30m)':'Attacker 킬(수/30분)',
 'attacker_win_rate':'Attacker 승률',
 'attacker_damage/30m' : 'Attacker 데미지(피해량/30분)',
 'defender_kill(30m)':'Defender 킬(수/30분)',
 'defender_win_rate':'Defender 승률',
 'defender_damage/30m':'Defender 데미지(피해량/30분)',
 'survival_rate':'생존율',
 'attacker_survival_rate':'Attacker 생존율',
 'defender_survival_rate' :'Defender 생존율',
 'num_of_attack/30m_점수':'사격(횟수/30분) 점수',
 'attacker_num_of_attack/30m_점수':'선공(횟수/30분) 점수',
 'defender_num_of_attack_점수':'Defender 교전 공격(횟수) 점수',
 'total_distance_점수':'총 이동거리 점수',
 'displacement_점수': '교전 이동거리 점수',
 'shooting_distance_점수': '사격거리 점수',
 '습득량_점수': '투척물 습득량 점수',
 '소비량_점수': '투척물 소비량 점수',
 '소비량/습득량_점수': '투척물 소비율 점수',
 'headshot_kills(30m)_점수': '헤드샷(킬수/30분) 점수',
 '확정킬_비율_점수' : '확정킬 비율 점수',
 'acc_점수':'명중률 점수',
 'attacker_acc_점수':'Attacker 명중률 점수',
 'defender_acc_점수':'Defender 명중률 점수',
 'kill(30m)_점수' : '킬(수/30분) 점수',
 'win_rate_점수' : '승률 점수',
 'damage/30m_점수' : '데미지(피해량/30분) 점수',
 'attacker_kill(30m)_점수':'Attacker 킬(수/30분) 점수',
 'attacker_win_rate_점수': 'Attacker 승률 점수',
 'attacker_damage/30m_점수' : 'Attacker 데미지(피해량/30분) 점수',
 'defender_kill(30m)_점수': 'Defender 킬(수/30분) 점수',
 'defender_win_rate_점수':'Defender 승률 점수',
 'defender_damage/30m_점수' : 'Defender 데미지(피해량/30분) 점수',
 'survival_rate_점수':'생존율 점수',
 'attacker_survival_rate_점수':'Attacker 생존율 점수',
 'defender_survival_rate_점수':'Defender 생존율 점수',
 'dbnos_점수':'DBNOs 점수',
 'time_survived(s)_점수':'생존시간 점수',
 '투척물_점수' : '투척물 점수',
 '거리_점수' : '거리 점수',
 '호전성_점수': '호전성점수',
 '운영_점수' : '운영 점수',
 '교전_전투력_점수' : '교전 전투력 점수',
 'attacker_전투력_점수':'Attacker 전투력 점수',
 'defender_전투력_점수':'Defender 전투력 점수' ,
 '전투_점수':'전투 점수',
 '사격_점수':'사격 점수',
 '생존력_점수':'생존력 점수'
}

# Summary

## 1. summary_2020

In [7]:
match_count = player_stats.groupby('player')['created_at'].count().to_frame().reset_index()
match_count.columns=['player', 'match_count']
match_count

,player,match_count
0,1IP_Huning,5
1,1IP_Jaemin,5
2,1IP_KidAsh,5
3,1IP_Lildara,5
4,303_Crissmazing,24
...,...,...
1084,slobs16,4
1085,slobs17,3
1086,slobs19,4
1087,slobs20,4


In [8]:
summary_2020_origin = player_stats.groupby('player').mean()

In [9]:
summary_2020 = summary_2020_origin.merge(match_count, how='left', on='player').sort_values('match_count', ascending=False).query("match_count>=20")

summary_2020.insert(loc=0, column='선수명', value=summary_2020.player.str.split('_').map(lambda x: '_'.join(x[1:])))
summary_2020.insert(loc=0, column='팀명', value=summary_2020.player.str.split('_').map(lambda x: x[0]))

for col in cols:
    to_point_10(summary_2020, col)

compute_summation(summary_2020)

summary_2020 = summary_2020.rename(columns=column_renames)
summary_2020 = summary_2020[['팀명', '선수명']+[col for col in column_renames.values() if col not in ['경기 시작 시간', '토너먼트ID']]+['최종 점수', '최종 순위'] ]
summary_2020

,팀명,선수명,선수ID,생존시간,DBNOs,총 이동거리,헤드샷 킬수,경기 순위,사격(횟수/30분),Defender 교전 공격(횟수),...,호전성점수,운영 점수,교전 전투력 점수,Attacker 전투력 점수,Defender 전투력 점수,전투 점수,사격 점수,생존력 점수,최종 점수,최종 순위
965,VLG,DAEVA,VLG_DAEVA,1122.688449,0.956522,4297.050589,0.210145,9.246377,81.273227,10.418011,...,6.333333,5.611111,5.875,5.500,6.000,5.791667,5.4,4.750,5.388194,276.0
14,4AM,Forever,4AM_Forever,1123.251855,1.224638,4474.574981,0.253623,8.340580,86.106599,9.568110,...,6.666667,5.666667,6.500,6.250,6.875,6.541667,6.0,5.250,5.864583,128.0
1075,iFTY,Jiaoyang,iFTY_Jiaoyang,1239.665754,0.811594,5677.809774,0.108696,7.644928,68.013986,9.236243,...,6.000000,5.444444,5.250,5.250,4.500,5.000000,5.0,5.250,5.173611,352.0
294,ENT,Seongjang,ENT_Seongjang,1284.197652,0.876812,5370.633468,0.166667,7.362319,52.983229,5.116154,...,5.000000,5.333333,5.500,5.750,4.250,5.166667,5.6,5.750,5.462500,255.0
295,ENT,Under,ENT_Under,1328.549841,1.231884,5482.094201,0.253623,7.362319,92.211068,8.413281,...,6.666667,6.722222,7.000,6.500,6.000,6.500000,5.8,6.250,6.318056,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,DIC,JJLeo,DIC_JJLeo,1183.856800,0.750000,4552.044480,0.050000,8.500000,39.149662,4.453704,...,3.833333,3.888889,3.625,4.375,1.750,3.250000,3.4,5.250,3.947222,637.0
78,APK,KIMGONI,APK_KIMGONI,1246.743700,0.300000,6905.435307,0.100000,7.700000,42.428303,6.468519,...,3.666667,4.111111,3.250,3.375,4.000,3.541667,4.2,4.750,4.150694,610.0
585,MSC,ELONIX,MSC_ELONIX,1411.648700,0.350000,6053.856456,0.000000,7.150000,31.636377,5.092857,...,3.000000,4.222222,3.625,3.625,4.500,3.916667,4.6,4.875,4.403472,560.0
268,E365,yzkeJ,E365_yzkeJ,1247.487450,0.600000,5107.567018,0.100000,10.200000,58.867975,6.127193,...,4.666667,4.500000,5.750,4.750,6.750,5.750000,6.5,5.250,5.500000,241.0


In [10]:
summary_2020.sort_values('최종 순위', ascending=True)

,팀명,선수명,선수ID,생존시간,DBNOs,총 이동거리,헤드샷 킬수,경기 순위,사격(횟수/30분),Defender 교전 공격(횟수),...,호전성점수,운영 점수,교전 전투력 점수,Attacker 전투력 점수,Defender 전투력 점수,전투 점수,사격 점수,생존력 점수,최종 점수,최종 순위
829,SQ,TGLTN,SQ_TGLTN,1397.261815,1.629630,4759.964248,0.407407,6.703704,146.988171,16.855151,...,10.000000,7.777778,9.250,8.000,8.125,8.458333,7.8,8.000,8.009028,1.0
995,WC,Sharpshot4k,WC_Sharpshot4k,1342.789500,1.950000,5410.485248,0.400000,7.000000,144.718084,18.750000,...,9.333333,8.000000,8.875,7.750,9.125,8.583333,7.1,7.750,7.858333,2.0
635,OATH,Snakers,OATH_Snakers,1405.261459,1.621622,6268.661470,0.229730,6.472973,114.666295,11.696714,...,9.166667,7.500000,8.750,8.000,5.875,7.541667,6.8,9.000,7.710417,3.0
559,MCG,Tanki995,MCG_Tanki995,1445.272958,1.583333,4509.232839,0.291667,6.458333,132.516502,11.208333,...,8.333333,7.111111,8.250,7.375,8.500,8.041667,6.8,8.500,7.613194,4.0
81,APK,maczora,APK_maczora,1302.044300,1.333333,6954.632010,0.200000,7.566667,83.109632,9.283333,...,6.000000,6.000000,7.625,6.500,8.000,7.375000,7.7,8.000,7.268750,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,FB,Kalimdor,FB_Kalimdor,1079.655875,0.416667,5170.322364,0.041667,8.666667,37.125184,2.686806,...,2.333333,3.333333,2.000,2.500,2.375,2.291667,2.5,2.750,2.718750,686.0
89,ATH,Lister,ATH_Lister,1049.834400,0.400000,5205.463988,0.050000,9.350000,41.064970,1.378333,...,2.833333,2.222222,2.750,2.500,3.500,2.916667,2.6,2.375,2.528472,687.0
991,WCS,Chaebin,WCS_Chaebin,1013.779950,0.150000,6112.864337,0.000000,10.300000,20.465457,0.879630,...,1.000000,1.722222,1.750,3.500,1.250,2.166667,3.1,1.875,2.215972,688.0
191,DA,SIXMO,DA_SIXMO,1034.014583,0.166667,4000.375668,0.000000,7.875000,39.396081,5.253623,...,3.500000,1.777778,1.000,2.000,0.625,1.208333,1.9,2.250,1.784028,689.0


In [11]:
# summary_2020['최종 점수'].sort_values(ascending=False).plot.hist()

# summary_2020['전투_점수'].sort_values(ascending=False)

# summary_2020['운영_점수'].sort_values(ascending=False)

# summary_2020['사격_점수'].sort_values(ascending=False)

# summary_2020['생존력_점수'].sort_values(ascending=False)

## 2. summary_2020_tournament

In [12]:
summary_2020_tournament = player_stats.groupby(['tournament_id', 'player']).mean().reset_index()
summary_2020_tournament['match_count'] = player_stats.groupby(['tournament_id', 'player'])['created_at'].count().values
summary_2020_tournament = summary_2020_tournament.query("match_count>=10").sort_values('match_count', ascending=False)

summary_2020_tournament.insert(loc=0, column='선수명', value=summary_2020_tournament.player.str.split('_').map(lambda x: '_'.join(x[1:])))
summary_2020_tournament.insert(loc=0, column='팀명', value=summary_2020_tournament.player.str.split('_').map(lambda x: x[0]))

for col in cols:
    to_point_10(summary_2020_tournament, col)

compute_summation(summary_2020_tournament)

summary_2020_tournament = summary_2020_tournament.rename(columns=column_renames)
summary_2020_tournament = summary_2020_tournament[['팀명', '선수명']+[col for col in column_renames.values() if col not in ['경기 시작 시간']]+['최종 점수', '최종 순위']]

summary_2020_tournament

,팀명,선수명,토너먼트ID,선수ID,생존시간,DBNOs,총 이동거리,헤드샷 킬수,경기 순위,사격(횟수/30분),...,호전성점수,운영 점수,교전 전투력 점수,Attacker 전투력 점수,Defender 전투력 점수,전투 점수,사격 점수,생존력 점수,최종 점수,최종 순위
674,KX,Cobra,cn-pcsa,KX_Cobra,1203.174989,1.266667,5132.525006,0.255556,8.555556,83.584274,...,6.000000,5.777778,7.000,6.250,6.000,6.416667,6.1,5.75,6.011111,308.0
701,LYG,MiGo1337,cn-pcsa,LYG_MiGo1337,1235.019089,0.900000,7053.205780,0.122222,8.788889,61.165809,...,5.333333,5.444444,5.750,6.250,4.500,5.500000,5.6,6.00,5.636111,501.0
698,LYG,Dec12th,cn-pcsa,LYG_Dec12th,1245.019989,1.011111,7250.191482,0.177778,8.788889,45.996221,...,4.666667,5.388889,5.750,5.500,6.000,5.750000,5.8,5.50,5.609722,514.0
764,RNG,TangDD,cn-pcsa,RNG_TangDD,1179.059422,0.777778,4545.333339,0.155556,8.500000,59.811854,...,5.333333,4.777778,5.500,5.500,5.250,5.416667,5.8,5.00,5.248611,762.0
805,WOL,067_Tezo,cn-pcsa,WOL_067_Tezo,1132.506822,1.100000,5051.372794,0.166667,9.322222,73.977275,...,6.000000,4.888889,5.750,5.750,4.750,5.416667,5.4,5.00,5.176389,802.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,PGA,AKA,as-pcs2kr,PGA_AKA,1090.116100,0.700000,6444.844734,0.200000,11.700000,59.259048,...,5.666667,3.611111,5.750,5.375,7.750,6.291667,5.8,6.25,5.488194,597.0
351,PGA,2mae,as-pcs2kr,PGA_2mae,978.679300,0.700000,7294.232609,0.000000,11.700000,20.041802,...,1.500000,2.444444,3.125,3.250,2.500,2.958333,4.1,1.00,2.625694,1486.0
348,PBN,OH9,as-pcs2kr,PBN_OH9,1398.940500,1.000000,6433.577396,0.500000,7.100000,62.490593,...,5.333333,5.222222,8.250,5.875,8.500,7.541667,8.6,6.75,7.028472,68.0
347,PBN,DParis,as-pcs2kr,PBN_DParis,1391.011800,1.100000,6735.446660,0.100000,7.100000,43.257587,...,4.333333,6.222222,5.000,5.500,1.625,4.041667,3.7,6.25,5.053472,877.0


## 3. summary_2020_match

In [14]:
summary_2020_match = player_stats.groupby(['tournament_id','created_at', 'player']).mean().reset_index().sort_values(['tournament_id', 'created_at'])
summary_2020_match.insert(loc=0, column='선수명', value=summary_2020_match.player.str.split('_').map(lambda x: '_'.join(x[1:])))
summary_2020_match.insert(loc=0, column='팀명', value=summary_2020_match.player.str.split('_').map(lambda x: x[0]))

for col in cols:
    to_point_10(summary_2020_match, col)
compute_summation(summary_2020_match)

summary_2020_match = summary_2020_match.rename(columns=column_renames)
summary_2020_match = summary_2020_match[['팀명', '선수명']+list(column_renames.values())+['최종 점수', '최종 순위']]
summary_2020_match['경기 시작 시간'] = pd.to_datetime(summary_2020_match['경기 시작 시간'])
summary_2020_match['경기 시작 시간'] = summary_2020_match['경기 시작 시간'].dt.strftime('%Y-%m-%d %H:%M')

summary_2020_match


,팀명,선수명,토너먼트ID,경기 시작 시간,선수ID,생존시간,DBNOs,총 이동거리,헤드샷 킬수,경기 순위,...,호전성점수,운영 점수,교전 전투력 점수,Attacker 전투력 점수,Defender 전투력 점수,전투 점수,사격 점수,생존력 점수,최종 점수,최종 순위
0,AFK,APXB,as-pcs1kr,2020-05-18 02:32,AFK_APXB,988.608,2,9350.177924,0,5,...,2.666667,3.000000,7.250,7.875,0.500,5.208333,4.6,6.375,4.795833,19041.0
1,AFK,Asag2,as-pcs1kr,2020-05-18 02:32,AFK_Asag2,1004.771,1,8779.109774,0,5,...,0.500000,2.611111,0.500,2.875,0.500,1.291667,2.4,2.375,2.169444,33862.0
2,AFK,Caley,as-pcs1kr,2020-05-18 02:32,AFK_Caley,1229.768,0,8621.725880,1,5,...,2.333333,2.166667,8.500,8.250,0.500,5.750000,5.4,5.625,4.735417,19415.0
3,AFK,Jaelong,as-pcs1kr,2020-05-18 02:32,AFK_Jaelong,814.769,0,8264.345427,0,5,...,0.500000,2.611111,0.500,2.875,0.500,1.291667,2.4,0.500,1.700694,37260.0
4,EZ,Donghyun,as-pcs1kr,2020-05-18 02:32,EZ_Donghyun,925.875,1,4549.920413,1,8,...,5.500000,3.111111,10.000,9.500,0.500,6.666667,5.8,6.625,5.550694,14369.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39413,VTR,Kamalz,sea-pcs2ap,2020-09-13 12:08,VTR_Kamalz,1892.271,0,8701.767600,0,2,...,1.166667,4.888889,6.375,5.000,4.000,5.125000,5.0,5.375,5.097222,17172.0
39414,XS,HOANGVU,sea-pcs2ap,2020-09-13 12:08,XS_HOANGVU,1900.364,2,7035.694100,0,1,...,8.000000,6.666667,8.500,6.500,10.000,8.333333,7.7,10.000,8.175000,1694.0
39415,XS,MeoU,sea-pcs2ap,2020-09-13 12:08,XS_MeoU,1907.488,6,8529.346200,0,1,...,9.833333,8.111111,8.250,7.625,8.250,8.041667,3.8,10.000,7.488194,3933.0
39416,XS,NgocTan,sea-pcs2ap,2020-09-13 12:08,XS_NgocTan,577.352,0,6093.167950,0,1,...,5.833333,3.888889,0.625,2.875,3.625,2.375000,3.1,0.500,2.465972,32042.0


# 저장

In [15]:
with pd.ExcelWriter('./data/results/선수평가.xlsx') as writer:
    summary_2020.to_excel(writer, sheet_name='2020 선수 평가', index=False)
    summary_2020_tournament.to_excel(writer, sheet_name='2020 토너먼트별 선수 평가', index=False)
    summary_2020_match.to_excel(writer, sheet_name='2020 경기별 선수 평가', index=False)    

- multi index
- sheet 자르기 and 순서바꾸기
- 점수 모델 좀 더 시도
- dashboard
- ppt
